### Init

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import math
import os
from tqdm import tqdm
from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rich
from rich import pretty, print
from glob import glob
from scipy import sparse
import pickle

plt.style.use("dark_background")
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 50
pd.options.display.max_rows = 50
pretty.install()

In [ ]:
def save_pickle(name, data):
    with open(name, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
train_df = pd.read_csv("data/stanford-ribonanza-rna-folding/train_data.csv")
test_df  = pd.read_csv("data/stanford-ribonanza-rna-folding/test_sequences.csv")

### Process BPP

In [ ]:
bpp_list = glob("data/stanford-ribonanza-rna-folding/Ribonanza_bpp_files/extra_data/*/*/*/*txt")
bpp_dict = {}
for i in range(0, len(bpp_list)):
    bpp_dict[bpp_list[i].split('/')[-1][:-4]] = bpp_list[i]
len(bpp_list)

In [ ]:
TRAIN_BPP_DIR = "data/stanford-ribonanza-rna-folding/train_sparse_bpps"
TEST_BPP_DIR = "data/stanford-ribonanza-rna-folding/test_sparse_bpps"
os.makedirs(TRAIN_BPP_DIR, exist_ok = True)
os.makedirs(TEST_BPP_DIR, exist_ok = True)


In [ ]:
def extract_bpp(df, save_path):
    len_arr = df.sequence.apply(len)
    seq_ids = df.sequence_id
    len_dict = {}
    
    for i in range(0, len(df)):
        len_dict[seq_ids[i]] = len_arr[i] 
        
    for i in tqdm(range(0, len(df))):
        len_seq = len_dict[seq_ids[i]]
        bpp_mat = np.zeros((len_seq, len_seq))
        f = open(bpp_dict[seq_ids[i]], 'r')
        L = f.readlines()
        f.close()
        for j in range(0, len(L)-1):
            tmp = L[j].split()
            row, col, bpp = int(tmp[0])-1, int(tmp[1])-1, float(tmp[2])
            bpp_mat[row, col] = bpp
        bpp_mat = sparse.csr_matrix(bpp_mat)
        save_pickle(f"{save_path}/{seq_ids[i]}.pkl", bpp_mat)



In [ ]:
extract_bpp(train_df, TRAIN_BPP_DIR)
extract_bpp(test_df,  TEST_BPP_DIR)

### Process dataframe

In [51]:
train_df

,sequence_id,sequence,experiment_type,dataset_name,reads,signal_to_noise,SN_filter,reactivity_0001,reactivity_0002,reactivity_0003,reactivity_0004,reactivity_0005,reactivity_0006,reactivity_0007,reactivity_0008,reactivity_0009,reactivity_0010,reactivity_0011,reactivity_0012,reactivity_0013,reactivity_0014,reactivity_0015,reactivity_0016,reactivity_0017,reactivity_0018,...,reactivity_error_0182,reactivity_error_0183,reactivity_error_0184,reactivity_error_0185,reactivity_error_0186,reactivity_error_0187,reactivity_error_0188,reactivity_error_0189,reactivity_error_0190,reactivity_error_0191,reactivity_error_0192,reactivity_error_0193,reactivity_error_0194,reactivity_error_0195,reactivity_error_0196,reactivity_error_0197,reactivity_error_0198,reactivity_error_0199,reactivity_error_0200,reactivity_error_0201,reactivity_error_0202,reactivity_error_0203,reactivity_error_0204,reactivity_error_0205,reactivity_error_0206
0,8cdfeef009ea,GGGAACGACUCGAGUAGAGUCGAAAAACGUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,2343,0.944,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51e61fbde94d,GGGAACGACUCGAGUAGAGUCGAAAAACAUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,5326,1.933,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25ce8d5109cd,GGGAACGACUCGAGUAGAGUCGAAAAACCUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,4647,2.347,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,07dcfb6d1965,GGGAACGACUCGAGUAGAGUCGAAAAACUUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,102843,11.824,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e561cc042a4c,GGGAACGACUCGAGUAGAGUCGAAAAACGAUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,7665,3.519,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643675,7951fb2f47f1,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,37530,7.248,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643676,e0dc5823e5e1,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,337248,17.902,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643677,0d6036529b42,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,44053,6.700,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643678,46d1f07d723b,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,108600,11.716,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
sample_train_df = pd.read_parquet('data/train_data_processed_ALL_2.parquet')

In [ ]:
# Consolidate reactivity and reactivity_error columns into single column arrays
tmp = train_df.filter(like="reactivity_error_")
reactivity_error = pd.Series(list(tmp.values))
train_df = train_df.drop(columns=tmp.columns)
tmp = train_df.filter(like="reactivity_")
reactivity = pd.Series(list(tmp.values))
train_df = train_df.drop(columns=tmp.columns)
train_df["error"] = reactivity_error
train_df["react"] = reactivity

In [ ]:
# Rename values and reorder columns for indexing
train_df["dataset_name"] = train_df["dataset_name"].apply(lambda x: x[:16] + x[20:] if x.endswith("EternaPlayers") else x[:-4])
train_df["experiment_type"] = train_df["experiment_type"].apply(lambda x: x[:3])
train_df = train_df[["sequence_id", "sequence", "dataset_name", "experiment_type", "reads", "signal_to_noise", "SN_filter", "react", "error"]]
train_df = train_df.rename(columns={"sequence_id": "seq_id", "sequence": "seq", "dataset_name": "dataset", "signal_to_noise": "SN"})

In [ ]:
# Index and unstack to consolidate DMS and 2A3 rows
df_train = df_train.set_index(["seq_id", "seq", "dataset", "experiment_type"])
df_train = df_train.unstack()
# display(df_train.head(5))
# print(df_train.columns.values) # (reads, 2A3), (reads, DMS)
df_train.columns = ['_'.join(c) for c in df_train.columns.values]
df_train = df_train.reset_index()

In [ ]:
df_train.head(1)

In [ ]:
# Remove trash sequences
df_train = df_train[(df_train.SN_2A3 > 0) | (df_train.SN_DMS > 0)]
df_train = df_train[(df_train.reads_2A3 > 0) | (df_train.reads_DMS > 0)]
nan_check = lambda x: np.isnan(x).all() == False
print(df_train.shape)
df_train = df_train[(df_train.react_2A3.apply(nan_check)) | (df_train.react_DMS.apply(nan_check))]
print(df_train.shape)
# Save
df_train.to_parquet("data/train_data_processed.parquet")

### Load

In [ ]:
df_train = pd.read_parquet("data/train_data_processed.parquet")
df_train

### EDA

In [ ]:
def tile_hist(df, labels, cols=4, sfigsize=(6, 4), log=True, bins=100):
    cols = len(labels) if len(labels) < cols else cols
    rows = math.ceil(len(labels) / cols)
    f, axs = plt.subplots(rows, cols, figsize=(sfigsize[0] * cols, sfigsize[1] * rows))
    axs = axs.flatten()
    for i in range(len(labels)):
        x = labels[i]
        b = len(np.unique(df[x].values.flatten()))
        axs[i].hist(df[x], label=x, log=log, bins=b if b < bins else bins)
        axs[i].legend(prop={"size": 8})
    plt.show()


hist_targets = [
    ["reads_2A3", "reads_DMS"],
    ["SN_2A3", "SN_DMS"],
    ["SN_filter_2A3", "SN_filter_DMS"],
]

tile_hist(df_train, hist_targets)